In [1]:
import org.apache.spark.sql.SparkSession
val sc = SparkSession.builder().getOrCreate()
sc.version

sc = org.apache.spark.sql.SparkSession@71822941


2.4.3

### 8. 조인 표현식

왼쪽, 오른쪽 데이터셋에 있는 하나 이상의 키값을 비교하고 왼쪽 데이터셋과 오른쪽 데이터셋의 결합 여부를 결정함 <br>

In [5]:
val person = Seq (
    (0, "Bill Chambers", 0, Seq(100)),
    (1, "Matei Zaharia", 1, Seq(500, 250, 100)),
    (2, "Michael Armbrust", 1, Seq(250, 100))
).toDF("id", "name", "graduate_program", "spark_status")

val graduateProgram = Seq (
    (0, "Masters", "School of Information", "UC Bereley"),
    (2, "Masters", "EECS", "UC Bereley"),
    (1, "Ph. D", "EECS", "UC Bereley")
).toDF("id", "degree", "department", "school")

val sparkStatus = Seq (
    (500, "Vice President") ,
    (250, "PMC Member"), 
    (100, "Contributor")
).toDF("id", "status")

person = [id: int, name: string ... 2 more fields]
graduateProgram = [id: int, degree: string ... 2 more fields]
sparkStatus = [id: int, status: string]


[id: int, status: string]

### 3. 내부조인


In [9]:
val joinExpression = person.col("graduate_program") === graduateProgram.col("id")

val wrongJoinExpression = person.col("name") === graduateProgram.col("school") // 두 키가 모두 존재하지 않음.

person.join(graduateProgram, joinExpression).show()

+---+----------------+----------------+---------------+---+-------+--------------------+----------+
| id|            name|graduate_program|   spark_status| id| degree|          department|    school|
+---+----------------+----------------+---------------+---+-------+--------------------+----------+
|  0|   Bill Chambers|               0|          [100]|  0|Masters|School of Informa...|UC Bereley|
|  2|Michael Armbrust|               1|     [250, 100]|  1|  Ph. D|                EECS|UC Bereley|
|  1|   Matei Zaharia|               1|[500, 250, 100]|  1|  Ph. D|                EECS|UC Bereley|
+---+----------------+----------------+---------------+---+-------+--------------------+----------+


joinExpression = (graduate_program = id)
wrongJoinExpression = (name = school)


(name = school)

In [11]:
var joinType = "inner" //조인 타입 설정
person.join(graduateProgram, joinExpression, joinType).show()

+---+----------------+----------------+---------------+---+-------+--------------------+----------+
| id|            name|graduate_program|   spark_status| id| degree|          department|    school|
+---+----------------+----------------+---------------+---+-------+--------------------+----------+
|  0|   Bill Chambers|               0|          [100]|  0|Masters|School of Informa...|UC Bereley|
|  2|Michael Armbrust|               1|     [250, 100]|  1|  Ph. D|                EECS|UC Bereley|
|  1|   Matei Zaharia|               1|[500, 250, 100]|  1|  Ph. D|                EECS|UC Bereley|
+---+----------------+----------------+---------------+---+-------+--------------------+----------+



joinType = inner


inner

### 4. 외부 조인
왼쪽, 오른쪽 Dataframe에 일치하는 로우가 없다면 스파크는 해당 위치에 null을 삽입함.

In [12]:
joinType = "outer"
person.join(graduateProgram, joinExpression, joinType).show()

+----+----------------+----------------+---------------+---+-------+--------------------+----------+
|  id|            name|graduate_program|   spark_status| id| degree|          department|    school|
+----+----------------+----------------+---------------+---+-------+--------------------+----------+
|   1|   Matei Zaharia|               1|[500, 250, 100]|  1|  Ph. D|                EECS|UC Bereley|
|   2|Michael Armbrust|               1|     [250, 100]|  1|  Ph. D|                EECS|UC Bereley|
|null|            null|            null|           null|  2|Masters|                EECS|UC Bereley|
|   0|   Bill Chambers|               0|          [100]|  0|Masters|School of Informa...|UC Bereley|
+----+----------------+----------------+---------------+---+-------+--------------------+----------+



joinType = outer


outer

### 5. 왼쪽 외부 조인
오른쪽 Dataframe에 일치하는 로우가 없다면 스파크는 해당 위치에 null을 삽입함.

In [13]:
joinType = "left_outer"
person.join(graduateProgram, joinExpression, joinType).show()

+---+----------------+----------------+---------------+---+-------+--------------------+----------+
| id|            name|graduate_program|   spark_status| id| degree|          department|    school|
+---+----------------+----------------+---------------+---+-------+--------------------+----------+
|  0|   Bill Chambers|               0|          [100]|  0|Masters|School of Informa...|UC Bereley|
|  1|   Matei Zaharia|               1|[500, 250, 100]|  1|  Ph. D|                EECS|UC Bereley|
|  2|Michael Armbrust|               1|     [250, 100]|  1|  Ph. D|                EECS|UC Bereley|
+---+----------------+----------------+---------------+---+-------+--------------------+----------+



joinType = left_outer


left_outer

### 6. 오른쪽 외부 조인
왼쪽 Dataframe에 일치하는 로우가 없다면 스파크는 해당 위치에 null을 삽입함.

In [14]:
joinType = "right_outer"
person.join(graduateProgram, joinExpression, joinType).show()

+----+----------------+----------------+---------------+---+-------+--------------------+----------+
|  id|            name|graduate_program|   spark_status| id| degree|          department|    school|
+----+----------------+----------------+---------------+---+-------+--------------------+----------+
|   0|   Bill Chambers|               0|          [100]|  0|Masters|School of Informa...|UC Bereley|
|null|            null|            null|           null|  2|Masters|                EECS|UC Bereley|
|   2|Michael Armbrust|               1|     [250, 100]|  1|  Ph. D|                EECS|UC Bereley|
|   1|   Matei Zaharia|               1|[500, 250, 100]|  1|  Ph. D|                EECS|UC Bereley|
+----+----------------+----------------+---------------+---+-------+--------------------+----------+



joinType = right_outer


right_outer

### 7. 왼쪽 세미 조인
오른쪽 Dataframe의 어떤 값도 포함하지 않기 때문에 다른 조인 타입과는 약간 다르다. <br>
단지 두번째 Dataframe은 값이 존재하는지 확인하기 위해 값만 비교하는 용도로 사용함. <br>
기존 조인 기능과는 달리 Dataframe의 필터 정보로 볼 수 있다.

In [15]:
joinType = "left_semi"

graduateProgram.join(person, joinExpression, joinType).show()

+---+-------+--------------------+----------+
| id| degree|          department|    school|
+---+-------+--------------------+----------+
|  0|Masters|School of Informa...|UC Bereley|
|  1|  Ph. D|                EECS|UC Bereley|
+---+-------+--------------------+----------+



joinType = left_semi


left_semi

### 8. 왼쪽 안티 조인
왼쪽 세미 조인의 반대 개념. 왼쪽 세미 조인처럼 오른쪽 Datafrmae의 어떤 값도 포함하지 않는다. <br>
SQL의 NOT IN과 같은 스타일의 필터이다.

In [16]:
joinType = "left_anti"

graduateProgram.join(person, joinExpression, joinType).show()

+---+-------+----------+----------+
| id| degree|department|    school|
+---+-------+----------+----------+
|  2|Masters|      EECS|UC Bereley|
+---+-------+----------+----------+



joinType = left_anti


left_anti

### 9. 자연 조인
조인하려는 컬럼을 암시적으로 추정. 암시적인 추정은 부정확한 결과를 만들어낼 수 있음.

### 10. 교차 조인 (카테시안 조인)
조건절을 기술하지 않는 내부 조인을 뜻함. 왼쪽 Datafrmae의 모든 로우를 오른쪽 Dataframe의 모든 로우와 결합함. <br>
엄청난 수의 로우를 가진 Dataframe이 생성됨. 1000개 로우가 존재하는 두 Dataframe은 1000*1000 개의 로우가 생성됨.

In [20]:
joinType = "cross"

graduateProgram.join(person, joinExpression, joinType).show() // id랑 graudate_program만 조합했음.

+---+-------+--------------------+----------+---+----------------+----------------+---------------+
| id| degree|          department|    school| id|            name|graduate_program|   spark_status|
+---+-------+--------------------+----------+---+----------------+----------------+---------------+
|  0|Masters|School of Informa...|UC Bereley|  0|   Bill Chambers|               0|          [100]|
|  1|  Ph. D|                EECS|UC Bereley|  2|Michael Armbrust|               1|     [250, 100]|
|  1|  Ph. D|                EECS|UC Bereley|  1|   Matei Zaharia|               1|[500, 250, 100]|
+---+-------+--------------------+----------+---+----------------+----------------+---------------+



joinType = cross


cross

In [18]:
person.crossJoin(graduateProgram).show()

+---+----------------+----------------+---------------+---+-------+--------------------+----------+
| id|            name|graduate_program|   spark_status| id| degree|          department|    school|
+---+----------------+----------------+---------------+---+-------+--------------------+----------+
|  0|   Bill Chambers|               0|          [100]|  0|Masters|School of Informa...|UC Bereley|
|  1|   Matei Zaharia|               1|[500, 250, 100]|  0|Masters|School of Informa...|UC Bereley|
|  2|Michael Armbrust|               1|     [250, 100]|  0|Masters|School of Informa...|UC Bereley|
|  0|   Bill Chambers|               0|          [100]|  2|Masters|                EECS|UC Bereley|
|  1|   Matei Zaharia|               1|[500, 250, 100]|  2|Masters|                EECS|UC Bereley|
|  2|Michael Armbrust|               1|     [250, 100]|  2|Masters|                EECS|UC Bereley|
|  0|   Bill Chambers|               0|          [100]|  1|  Ph. D|                EECS|UC Bereley|


### 11. 조인 사용 시 문제점

In [21]:
// 복합 데이터 타입의 조인
// sparkStatus의 id와 person의 spark_status를 알아서 매칭
import org.apache.spark.sql.functions.expr

person.withColumnRenamed("id", "personId")
    .join(sparkStatus, expr("array_contains(spark_status, id)")).show()

+--------+----------------+----------------+---------------+---+--------------+
|personId|            name|graduate_program|   spark_status| id|        status|
+--------+----------------+----------------+---------------+---+--------------+
|       0|   Bill Chambers|               0|          [100]|100|   Contributor|
|       1|   Matei Zaharia|               1|[500, 250, 100]|500|Vice President|
|       1|   Matei Zaharia|               1|[500, 250, 100]|250|    PMC Member|
|       1|   Matei Zaharia|               1|[500, 250, 100]|100|   Contributor|
|       2|Michael Armbrust|               1|     [250, 100]|250|    PMC Member|
|       2|Michael Armbrust|               1|     [250, 100]|100|   Contributor|
+--------+----------------+----------------+---------------+---+--------------+



In [26]:
// 중복 컬럼명 처리
// 1. 조인에 사용할 Dataframe의 특정 키가 동일한 이름을 가지며, 키가 제거되지 않도록 조인 표현식에 명시하는 경우
// 2. 조인 대상이 아닌 두 개의 컬럼이 동일한 이름을 가진 경우

// 잘못된 경우
val gradProgramDupe = graduateProgram.withColumnRenamed("id", "graduate_program")
val joinExpr = gradProgramDupe.col("graduate_program") === person.col("graduate_program")

person.join(gradProgramDupe, joinExpr).show() // 두개의 graduate_program이 존재

//person.join(gradProgramDupe, joinExpr).select("graduate_program").show() // 선택시 에러가 발생함.

+---+----------------+----------------+---------------+----------------+-------+--------------------+----------+
| id|            name|graduate_program|   spark_status|graduate_program| degree|          department|    school|
+---+----------------+----------------+---------------+----------------+-------+--------------------+----------+
|  0|   Bill Chambers|               0|          [100]|               0|Masters|School of Informa...|UC Bereley|
|  2|Michael Armbrust|               1|     [250, 100]|               1|  Ph. D|                EECS|UC Bereley|
|  1|   Matei Zaharia|               1|[500, 250, 100]|               1|  Ph. D|                EECS|UC Bereley|
+---+----------------+----------------+---------------+----------------+-------+--------------------+----------+



gradProgramDupe = [graduate_program: int, degree: string ... 2 more fields]
joinExpr = (graduate_program = graduate_program)


lastException: Throwable = null


(graduate_program = graduate_program)

In [30]:
// 1. 해결 방법: 다른 조인 표현식 사용 (자동으로 하나의 컬럼이 제거됨))
person.join(gradProgramDupe, "graduate_program").select("graduate_program").show()

+----------------+
|graduate_program|
+----------------+
|               0|
|               1|
|               1|
+----------------+



In [29]:
// 2. 해결 방법: 조인 후 컬럼 제거
person.join(gradProgramDupe, joinExpr).drop(person.col("graduate_program")).select("graduate_program").show()

+----------------+
|graduate_program|
+----------------+
|               0|
|               1|
|               1|
+----------------+



In [31]:
// 3. 해결 방법: 조인전 컬럼명 변경
val gradProgram3 = graduateProgram.withColumnRenamed("id", "grad_id")
val joinExpr = person.col("graduate_program") === gradProgram3.col("grad_id")
person.join(gradProgram3, joinExpr).show()

+---+----------------+----------------+---------------+-------+-------+--------------------+----------+
| id|            name|graduate_program|   spark_status|grad_id| degree|          department|    school|
+---+----------------+----------------+---------------+-------+-------+--------------------+----------+
|  0|   Bill Chambers|               0|          [100]|      0|Masters|School of Informa...|UC Bereley|
|  2|Michael Armbrust|               1|     [250, 100]|      1|  Ph. D|                EECS|UC Bereley|
|  1|   Matei Zaharia|               1|[500, 250, 100]|      1|  Ph. D|                EECS|UC Bereley|
+---+----------------+----------------+---------------+-------+-------+--------------------+----------+



gradProgram3 = [grad_id: int, degree: string ... 2 more fields]
joinExpr = (graduate_program = grad_id)


(graduate_program = grad_id)

### 12. 스파크의 조인 수행 방식
두 가지 핵심 전략을 이해해야 한다.
- 노드간 네트워크 통신 전략
- 노드별 연산 전략

### 12.1. 노드간 네트워크 통신 전략

조인은 두 가지 클러스터 통신 방식을 활용한다. 전체 노드간 통신을 유발하는 **셔플 조인**과 그렇지 않으면 **브로드캐스트 조인**. <br>

큰 테이블과 큰 테이블 조인 (책에 그림 참고)
- 셔플 조인은 전체 노드간 통신이 발생하며, 많은 네트워크 자원을 사용한다.
- 즉, 전체 조인 프로세스가 진행되는 동안 모든 워커 노트에서 통신이 발생함.

큰 테이블과 작은 테이블 조인
- 조인 연산을 최적화 할 수 있다. 브로드캐스트 조인
- 작은 dataframe을 전체 워커 노드에 복제하는 방법. 이렇게 하면 자원을 많이 사용하는 것처럼 보이지만 조인 내 전체 노드가 통신하는 현상을 방지함.
- 브로드캐스트 조인도 이전 방식과 마찬가지로 노드 간 통신이 발생하지만, 노드 사이에 추가적인 통신은 발생하지 않는다.
- 단점으로는 너무 큰 데이터를 브로드캐스트 하면 고비용의 수집 연산이 발생하므로, 드라이버 노드가 비정상적으로 종료될 수 있다.

In [32]:
val joinExpr = person.col("graduate_program") === graduateProgram.col("id")
person.join(graduateProgram, joinExpr).explain()

== Physical Plan ==
*(1) BroadcastHashJoin [graduate_program#62], [id#77], Inner, BuildLeft
:- BroadcastExchange HashedRelationBroadcastMode(List(cast(input[2, int, false] as bigint)))
:  +- LocalTableScan [id#60, name#61, graduate_program#62, spark_status#63]
+- LocalTableScan [id#77, degree#78, department#79, school#80]


joinExpr = (graduate_program = id)


(graduate_program = id)

In [33]:
import org.apache.spark.sql.functions.broadcast

val joinExpr = person.col("graduate_program") === graduateProgram.col("id")
person.join(broadcast(graduateProgram), joinExpr).explain()

== Physical Plan ==
*(1) BroadcastHashJoin [graduate_program#62], [id#77], Inner, BuildRight
:- LocalTableScan [id#60, name#61, graduate_program#62, spark_status#63]
+- BroadcastExchange HashedRelationBroadcastMode(List(cast(input[0, int, false] as bigint)))
   +- LocalTableScan [id#77, degree#78, department#79, school#80]


joinExpr = (graduate_program = id)


(graduate_program = id)